In [16]:
import cv2
import time
import os
from pathlib import Path

import concurrent
import requests

In [17]:
def get_meta(img: cv2.Mat):
    image_data = cv2.imencode('.jpg', img)[1].tobytes()
    urls = [
        
        "https://72c4-34-143-252-179.ngrok.io/useColor",
        "https://52c7-35-247-181-246.ngrok.io/predict"
    ]

    def request_post(url, data):
        return requests.post(url, files={"image": data}).json()

    with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
        res = [executor.submit(request_post, url, image_data) for url in urls]
        concurrent.futures.wait(res)

    return {"color": res[0].result()["predicted"], "model": res[1].result()["predicted"]}

In [8]:
def crop_do (coor, img, conf, cls):
    current_time = int(round(time.time() * 1000))
    print(current_time)
    if not os.path.exists("./runs/"):
        os.makedirs("./runs/")
    Path("./runs/Use1/").mkdir(parents=True, exist_ok=True)
    crop_img = img[ int(coor[1]) : int(coor[3]), int(coor[0]): int(coor[2])]
    #result = cv2.resize(crop_img, (640, 640))
    #result = cv2.cvtColor(result , cv2.COLOR_RGB2GRAY)
    meta = get_meta(crop_img)
    print(meta)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, f'Color: {meta["color"]}', (10,80), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(img, f'Model: {meta["model"]}', (10,160), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imwrite("./runs/Use1/img" + str(current_time) + ".jpg", img)

In [19]:
from lean_detect import UseModel
veh = UseModel("yolov7.pt", detect_class=[2, 5, 7], confident=0.75)

YOLOR 🚀 2022-12-7 torch 1.12.1 CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 306 layers, 36905341 parameters, 36905341 gradients
/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [10]:
veh.detect("./clips/use001.MP4" , do_function=crop_do)

1676910856195
{'color': 'Black', 'model': 'Mazda2'}
1676910878825
{'color': 'White', 'model': 'yaris_old'}
1676910890839
{'color': 'Grey', 'model': 'unknown'}
1676910917750
{'color': 'Bronze', 'model': 'e150'}
1676910956913
{'color': 'Black', 'model': 'unknown'}
1676910972707
{'color': 'Blue', 'model': 'unknown'}
1676910984259
{'color': 'Brown', 'model': 'acv70'}
1676911044339
{'color': 'Black', 'model': 'yaris_mama'}
1676911066751
{'color': 'Black', 'model': 'civic_fc'}
1676911078366
{'color': 'Black', 'model': 'unknown'}
1676911153672
{'color': 'Blue', 'model': 'e160'}
1676911188745
{'color': 'White', 'model': 'rt50'}
1676911225722
{'color': 'Grey', 'model': 'unknown'}
1676911246312
{'color': 'Grey', 'model': 'unknown'}
1676911267000
{'color': 'Grey', 'model': 'unknown'}
1676911281245
{'color': 'Black', 'model': 'Mazda2'}
1676911288581
{'color': 'Black', 'model': 'unknown'}
1676911293145
{'color': 'Black', 'model': 'unknown'}
1676911295670
{'color': 'Brown', 'model': 'unknown'}
16769

# Google storage

In [5]:
! pip install google-cloud
! pip install pymongo

  Using cached google_cloud-0.34.0-py2.py3-none-any.whl (1.8 kB)


In [2]:
from google.cloud import storage
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'./carrgclassification-857e3c375cdd.json'

def upload_cs_file(bucket_name, source_file_name, destination_file_name): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(destination_file_name)
    blob.upload_from_filename(source_file_name)

In [7]:
import datetime

def get_cs_file_url(bucket_name, file_name, expire_in=datetime.datetime.now() + datetime.timedelta(4000)): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    url = bucket.blob(file_name).generate_signed_url(expire_in)

    return url

In [3]:
upload_cs_file('images-bucks', "./DemoCase/img1676910856195.jpg", "test.jpg")

In [10]:
url = get_cs_file_url('images-bucks', "test.jpg")
url

'https://storage.googleapis.com/images-bucks/test.jpg?Expires=2022666100&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=cvXJubE%2BArPl2Z%2FrgmTN8pIU8w54gBJgVjd2roEXdUrm%2B2sbWpjVj%2FPF5R%2FFlXepsrQk4umm1gJ2Fm2qWi8nHW8XGyXoijH24%2F4p70ouiIj9UrS4M9twTlR0dEpb6w%2FAEyTWZK9yU2irtFpVClltdWgDA4TCTt8YBVSg5g%2BbRqaX5y3wlPRV16fdfqaZGzEVrYHtSn%2B8V%2FuCR2097ymAYBpOuSOYTl1hP6e07EprNFqC4ALQ3vP%2B3HFvXTVkC0ZziviE54m%2FIa1SaHUuSgPyYpCxahOAbYNBR6mcbsuo57oLQ9fEqhugVzjENkohyEEmXWcR9wkYORLDiRRUFHokjQ%3D%3D'

In [20]:
import database as db

In [21]:
import json
 
# Opening JSON file
with open('car_meta.json') as json_file:
    car_meta = json.load(json_file)

In [23]:
def cloud_image (bucket_name, source_file_name, destination_file_name):
    upload_cs_file(bucket_name, source_file_name, destination_file_name)
    url = get_cs_file_url(bucket_name, destination_file_name)
    return url


In [24]:
def crop_do (coor, img, conf, cls):
    current_time = int(round(time.time() * 1000))
    print(current_time)
    if not os.path.exists("./runs/"):
        os.makedirs("./runs/")
    Path("./runs/Use1/").mkdir(parents=True, exist_ok=True)
    crop_img = img[ int(coor[1]) : int(coor[3]), int(coor[0]): int(coor[2])]
    #result = cv2.resize(crop_img, (640, 640))
    #result = cv2.cvtColor(result , cv2.COLOR_RGB2GRAY)
    meta = get_meta(crop_img)
    print(meta)
    car_detail = car_meta[meta["model"]]
    print(car_detail)
    cv2.imwrite(f"./runs/Use1/{current_time}.jpg", img)
    origin_img_path = cloud_image('images-bucks', f"./runs/Use1/{current_time}.jpg", f'{current_time}-origin.jpg')
    cv2.imwrite(f"./runs/Use1/{current_time}-crop.jpg", crop_img)
    crop_img_path = cloud_image('images-bucks', f"./runs/Use1/{current_time}-crop.jpg", f'{current_time}-crop.jpg')

    db.update_data(car_detail["make"], car_detail["model"], car_detail["class"], "Unknown", meta["color"], origin_img_path, None, crop_img_path)
    

In [22]:
veh.detect("./clips/use001.MP4" , do_function=crop_do)

Exception: ERROR: /Users/thisisthanick/Library/Mobile Documents/com~apple~CloudDocs/Dev/CarDetection_WebApp/clips/use001.MP4 does not exist

In [2]:
from detect_flow import detect_flow

YOLOR  3b7f6fc torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191.375MB)



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


c:\Users\PPUNN\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 306 layers, 36905341 parameters, 36905341 gradients, 104.5 GFLOPS
c:\Users\PPUNN\anaconda3\lib\site-packages\torch\nn\modules\module.py:561: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  if param.grad is not None:


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [3]:
detect_flow("./clips/use001.MP4")

1677833926199
{'color': 'Black', 'model': 'Mazda2', 'possible': [{'class': 'Mazda2', 'conf': 0.7645111680030823}, {'class': 'corolla_cross', 'conf': 0.023793384432792664}, {'class': 'acv40', 'conf': 0.022760875523090363}, {'class': 'e160b', 'conf': 0.020936021581292152}, {'class': 'e150', 'conf': 0.019416043534874916}], 'plate_num': '7กล838', 'plate_url': 'https://storage.googleapis.com/images-bucks/16778339282655-plate.jpg?Expires=2023381145&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=HPwRrcueE%2BIjforfSOPvae2j%2FMPUkUJIgY6TW5HqM9nvtRNPdlqyOwnE4PmKb%2BopVWg9pl%2FT%2FNO4tJIBP11wkuZoUEv1z10p7GPy0Su23T8xXDGMV6gMCXNbJdj5qrZ1c%2BBrpB6WVJBkTix0i8e5FMAuHrRuwuJsulGzfFKHicXxNLD4PvriDIkTaRHC8TKmxW2Osb6%2BAFFaUk7jVgkhtJSoJhyIR7XE15bchpJ3F%2BnI0r0QRwN4VFV0018R8o5B6MsmUq487EHkzIyEnDc%2FDqMypLD%2Fcrx%2FLwGGJ0urfK5gV5IgL1%2FRSWzOLlRPN4x8%2BW2ExssjkgElKAkG3K6KHg%3D%3D'}
{'make': 'Mazda', 'model': 'Mazda 2', 'class': 'Eco', 'code': 'DL'}
1677833952598
{'color':

In [4]:
detect_flow("./clips/GX030043.MP4")

1677835112439
{'color': 'Black', 'model': 'unknown', 'possible': [{'class': 'Mazda2', 'conf': 0.21225593984127045}, {'class': 'e160b', 'conf': 0.12583404779434204}, {'class': 'corolla_cross', 'conf': 0.07256624102592468}, {'class': 'e150', 'conf': 0.07010424137115479}, {'class': 'city_jimmy', 'conf': 0.06665734946727753}], 'plate_num': 'ฌ5', 'plate_url': 'https://storage.googleapis.com/images-bucks/16778351144119-plate.jpg?Expires=2023381145&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=kkzFc5XNayeeZPvUYGrzmKjzlfjI7pp3e%2FuWTX9esqAknuAA4c0l85Q3mUStncwHWggraL%2BH4TDOYTroTVI4yB21MHOtdHkgWsafq1%2FqbHLyfS8lmBJEbPnt7qs5Hstohy4c0SsM3APpFuLlCVQXs559c1Q3PII5izGmObzRAIpzeRI1D%2FKYrXDXC6ySsYY2QMZ3%2FnteelaIZZrftiEa1PCPMVl9wbuqihV9MeHCPFn7SVlRB5ftQTFOiFJFSfGGV%2FEXxO6Jepb8Gw6GY%2FAp3vh9fohvTU9TArP5dVi%2FBQCv5O7wMVV6WqLNpPYawkMJCdOniXiY0DpW3FPE0qTNjQ%3D%3D'}
{'make': 'ไม่สามารถระบุแบรนด์', 'model': 'ไม่สามารถระบุรุ่น', 'class': 'ไม่สามารถระบุรุ่น', 'code': '0

In [3]:
detect_flow("./clips/GX040043.MP4")

1677836591723
{'color': 'Black', 'model': 'rt50', 'possible': [{'class': 'rt50', 'conf': 0.30642005801200867}, {'class': 'civic_fc', 'conf': 0.11942697316408157}, {'class': 'city_mawin', 'conf': 0.06107889488339424}, {'class': 'corolla_cross', 'conf': 0.055155619978904724}, {'class': 'Mazda2', 'conf': 0.05221147835254669}], 'plate_num': '1ก51818', 'plate_url': 'https://storage.googleapis.com/images-bucks/16778365938116-plate.jpg?Expires=2023381145&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=SPGwHbWrHhzcYujfJ9vVqlK1QEJuQt8wJ4Lq5NIzK6Meb%2FX6H1B49K%2BwmI88%2Be39GK3eX6pilQFjJL805CrX7sw2swvhHlVV1u7Nr9MmCj%2FRwWTttTFRAm%2Bto3U6LGPi2vBaoq8i1rnMtFICV5H%2BiyTuX6oIEf4xLaZBKDm05sLSdvHOKpl6jZIXVOMhbndGTIIuu18WOjhTxHXluBiu4lEFAJfypWYy6pO2ZEQ7GCL8ns0ip9Mc6SRTHGkDWLVxvgOXGjVDcVBXxCL4Ob3xL3mdw8Iv%2BM7yG0hUuueGfWuBNJKz4n8z6l0g166PV3uwCu91N4H%2FJKy439g9L37qiQ%3D%3D'}
{'make': 'Isuzu', 'model': 'D-max', 'class': 'Pickup', 'code': 'rt50'}
1677836598897
{'color': '

In [4]:
detect_flow("./clips/use002.MP4")

1677837259212
{'color': 'White', 'model': 'unknown', 'possible': [{'class': 'acv40', 'conf': 0.21909217536449432}, {'class': 'Mazda2', 'conf': 0.11222092807292938}, {'class': 'corolla_cross', 'conf': 0.09528390318155289}, {'class': 'civic_fc', 'conf': 0.09000787883996964}, {'class': 'e160b', 'conf': 0.08466839045286179}], 'plate_num': '4กจ61', 'plate_url': 'https://storage.googleapis.com/images-bucks/16778372613682-plate.jpg?Expires=2023381145&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=a6Ksb99UWjQ%2ByB23R16xJc81Bam4Ujf0hCn3D8TdiYR1ER5WIQwI97mF5dGq%2Fyk0KwVPm2HKs%2Feko%2F5GyMd1%2BmwwAqwpklwsy72hlmiKg8PXP38ecyZpkb9BZz2jAkLzIsB%2B3b8CLP5T7d71X2yR6lFeZZSLbVTCZlJlaBxVScvONEcYeS2SgDQbq22QP9RuQJ43w%2FEjOMxbqLpujPfkSKx0CJvHEt9eMKBfOZEc%2BBR3QzcGyFgfxeH5dAPMXa%2FD1dVZBS%2B6k13b7ZfIRfY%2FriqWtKC6lnDTVHNfne1loe00DVO9F5JDm6SGGYvEo%2Fr4KbJl4%2FlMjfkCkIpv58oO7w%3D%3D'}
{'make': 'ไม่สามารถระบุแบรนด์', 'model': 'ไม่สามารถระบุรุ่น', 'class': 'ไม่สามารถระบุรุ่น'

In [5]:
detect_flow("./clips/use003.MP4")

1677838502550
{'color': 'Black', 'model': 'unknown', 'possible': [{'class': 'corolla_cross', 'conf': 0.1765909194946289}, {'class': 'Mazda2', 'conf': 0.10342595726251602}, {'class': 'acv70', 'conf': 0.08111449331045151}, {'class': 'rt50', 'conf': 0.07470075786113739}, {'class': 'yaris_old', 'conf': 0.07000228017568588}], 'plate_num': '5กธ5531', 'plate_url': 'https://storage.googleapis.com/images-bucks/16778385046779-plate.jpg?Expires=2023381145&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=B7W1vQjOYbmCNQG218SoEXroFcCzH9cxPWJWQ9lUR81DWQOGl8CQluB2bS88mOXxXc1xdUlYn2KhKjnkqR3GJ51Vwr%2FLZzKqEZ%2B%2FKVD69ANtlH4bxERcAdfQRBbRjcQofhchvtAzM6ZiGaNlLoR0I8ojdFthZEhJyZYlETs5kysM7vfnZFu%2FS4RRbzZPOlpKHqDKd8mPjOKfLifYQopALOdpHzcjYYzpozGNbYbCmGI4K3ZJQMdg%2BTtCurK0ITuXEJLqxwWD8vcaX8iNq5XeotQUY6RNs4R1tYQgLt4VRrJL47PnR7ldjnVYWPK%2FtL%2BNsY%2BRmWhfoob2WmoUj0LXEA%3D%3D'}
{'make': 'ไม่สามารถระบุแบรนด์', 'model': 'ไม่สามารถระบุรุ่น', 'class': 'ไม่สามารถระบุรุ่น', 'code':

In [6]:
detect_flow("./clips/use004.MP4")

1677839939744
{'color': 'Black', 'model': 'revo', 'possible': [{'class': 'revo', 'conf': 0.6163638830184937}, {'class': 'e160', 'conf': 0.05726507678627968}, {'class': 'corolla_cross', 'conf': 0.04298018291592598}, {'class': 'city_mawin', 'conf': 0.03908521309494972}, {'class': 'city_jimmy', 'conf': 0.030950477346777916}], 'plate_num': '3256887', 'plate_url': 'https://storage.googleapis.com/images-bucks/16778399418064-plate.jpg?Expires=2023381145&GoogleAccessId=image-buck-sa%40carrgclassification.iam.gserviceaccount.com&Signature=sW1VXxIKtY2mLOtym3vrG1YbcAeB5ICfXUgQSAW6qP8qixmOguesN%2B5ez32t20niJSBKjXpm8JN3oflB3igUw616tTwPI%2FHQPnJ0M04Mx6G4QxdMsaZFmBlzr1CQC%2BFtWFOXc%2BCQojmhQNregSbFQIeiU1Nubp3cPy%2BJEXpU6Klv0IiQTy3yrCXRMRCgufr77i4oeKWAZyjA9SKeKt%2FPT2FSTeLwH08xFIlofQljWDzm514vy1t%2FDgh9vW8TNJuJNA6A4LAurqblGTMTS5Zpya%2FTl1tOfEDAhHmYfAVar0WvJCzebiRdn74RLtaevPDbXiOdN9GDdJJqJsZurnGWvw%3D%3D'}
{'make': 'Toyota', 'model': 'Hilux Revo', 'class': 'Pickup', 'code': 'an120'}
1677839945147
{'col

In [7]:
detect_flow("./clips/use005.MP4")

1677841347334
{'color': 'White', 'model': 'unknown', 'possible': [{'class': 'civic_fc', 'conf': 0.1131572425365448}, {'class': 'rt50', 'conf': 0.10683762282133102}, {'class': 'e160', 'conf': 0.10399185121059418}, {'class': 'city_mawin', 'conf': 0.08740764111280441}, {'class': 'acv40', 'conf': 0.08511590957641602}], 'plate_num': 'ไม่พบป้ายทะเบียน', 'plate_url': None}
{'make': 'ไม่สามารถระบุแบรนด์', 'model': 'ไม่สามารถระบุรุ่น', 'class': 'ไม่สามารถระบุรุ่น', 'code': '00'}
1677841351644
{'color': 'Grey', 'model': 'unknown', 'possible': [{'class': 'rt50', 'conf': 0.13135729730129242}, {'class': 'civic_fc', 'conf': 0.10641234368085861}, {'class': 'acv40', 'conf': 0.09361694008111954}, {'class': 'corolla_cross', 'conf': 0.08862750977277756}, {'class': 'Mazda2', 'conf': 0.07897607982158661}], 'plate_num': 'ไม่พบป้ายทะเบียน', 'plate_url': None}
{'make': 'ไม่สามารถระบุแบรนด์', 'model': 'ไม่สามารถระบุรุ่น', 'class': 'ไม่สามารถระบุรุ่น', 'code': '00'}
1677841355451
{'color': 'White', 'model': 'un

In [1]:
! python app.py

^C
